<a href="https://colab.research.google.com/github/shakurovas/pt-ms_gb/blob/master/homework7_pt%26ms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
# from sympy import diff # для производных
# from sympy.abc import x, y # для производных

**Задание 1:** Даны значения величины заработной платы заемщиков банка (zp) и значения их поведенческого кредитного скоринга (ks): zp = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110], ks = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832]. Используя математические операции, посчитать коэффициенты линейной регрессии, приняв за X заработную плату (то есть, zp - признак), а за y - значения скорингового балла (то есть, ks - целевая переменная). Произвести расчет как с использованием intercept, так и без.

**Решение:**

In [2]:
zp = np.array([35, 45, 190, 200, 40, 70, 54, 150, 120, 110])
reshaped_zp = zp.reshape((-1, 1))
ks = np.array([401, 574, 874, 919, 459, 739, 653, 902, 746, 832])

По формулам:

In [3]:
b = (np.mean(zp * ks) - np.mean(zp) * np.mean(ks)) / (np.mean(zp**2) - np.mean(zp) ** 2)
b

2.620538882402765

In [4]:
a = np.mean(ks) - b * np.mean(zp)
a

444.1773573243596

In [5]:
r = b * np.std(zp) / np.std(ks)
R = r ** 2
R

0.7876386635293682

Это означает, что **78.8%** вариации поведенческого кредитного скоринга заёмщиков банка(y) объясняется вариацией фактора x  — их заработной платой

In [6]:
ks_pred = a + b * zp
ks_pred

array([535.89621821, 562.10160703, 942.07974498, 968.2851338 ,
       548.99891262, 627.61507909, 585.68645697, 837.25818968,
       758.64202321, 732.43663439])

Средняя ошибка аппроксимации:

In [7]:
A_mean = 100 * np.mean(np.abs((ks - ks_pred) / ks))
A_mean

11.46925184356171

Т. к. это значение немного больше 10%, то можем сказать, что модель достаточно неплохо описывает эмпирические данные, но можно и лучше:)

Вариант решения через матричный метод:

In [8]:
zp_for_matrix = np.hstack([np.ones((zp.shape[0], 1)), reshaped_zp])
zp_for_matrix

array([[  1.,  35.],
       [  1.,  45.],
       [  1., 190.],
       [  1., 200.],
       [  1.,  40.],
       [  1.,  70.],
       [  1.,  54.],
       [  1., 150.],
       [  1., 120.],
       [  1., 110.]])

С интерцептом:

In [9]:
b = np.dot(np.linalg.inv(np.dot(zp_for_matrix.T, zp_for_matrix)), zp_for_matrix.T)
b = np.dot(b, ks)
b

array([444.17735732,   2.62053888])

Без интерцепта:

In [10]:
no_intercept_b = np.dot(np.linalg.inv(np.dot(reshaped_zp.T, reshaped_zp)), reshaped_zp.T)
no_intercept_b = np.dot(no_intercept_b, ks)
no_intercept_b

array([5.88982042])

Проверим также через встроенные функции библиотеки sklearn:

In [11]:
model = LinearRegression().fit(reshaped_zp, ks) # Эта операция создаёт переменную model в качестве экземпляра LinearRegression и с помощью .fit() вычисляются оптимальные значение весов B1, B0
print(model)

LinearRegression()


In [12]:
r_sq = model.score(reshaped_zp, ks)
print('coefficient of determination:', r_sq)

coefficient of determination: 0.7876386635293685


In [13]:
print('intercept:', model.intercept_) # одно значение, т. к. интерцепт всегда один
print('slope:', model.coef_) # массив, т. к. этих значений мб несколько

intercept: 444.1773573243595
slope: [2.62053888]


То есть,  ***y = 444.177 + x * 2.621*** - если считать с интерцептом

А если без интерцепта, то получится следующее:

In [14]:
no_intercept_model = LinearRegression(fit_intercept=False).fit(reshaped_zp, ks)
print(no_intercept_model)

LinearRegression(fit_intercept=False)


In [15]:
r_sq_no_intercept = no_intercept_model.score(reshaped_zp, ks)
print('coefficient of determination:', r_sq_no_intercept)

coefficient of determination: -0.8549037531632884


In [16]:
print('intercept:', no_intercept_model.intercept_)
print('slope:', no_intercept_model.coef_)

intercept: 0.0
slope: [5.88982042]


То есть,  ***y = x * 5.890*** - если без интерцепта

**Задание 2:** Посчитать коэффициент линейной регрессии при заработной плате (zp), используя градиентный спуск (без intercept).

**Решение:**

In [17]:
alpha = 10e-6
B1 = 0.1
n = zp.shape[0]
print(alpha, B1, n)

1e-05 0.1 10


In [20]:
def mse(B1, y=ks, X=zp, n=n):
  return np.sum((B1*X - y) ** 2) / n

In [22]:
for i in range(50):
  B1 -= alpha * (2 / n) * np.sum((B1 * zp - ks) * zp)
  print(f'Iteration: {i}, B1={B1}, mse={mse(B1)}')

Iteration: 0, B1=5.865666277993195, mse=56524.89599526941
Iteration: 1, B1=5.872321517085174, mse=56521.07697055634
Iteration: 2, B1=5.877143024839662, mse=56519.072540606314
Iteration: 3, B1=5.880636052919541, mse=56518.0205077638
Iteration: 4, B1=5.883166639986514, mse=56517.468344241344
Iteration: 5, B1=5.884999969337749, mse=56517.17853907253
Iteration: 6, B1=5.88632815778618, mse=56517.02643370434
Iteration: 7, B1=5.887290387815039, mse=56516.94660061122
Iteration: 8, B1=5.887987492679586, mse=56516.9046999023
Iteration: 9, B1=5.888492522846594, mse=56516.88270815252
Iteration: 10, B1=5.8888584010416265, mse=56516.87116569725
Iteration: 11, B1=5.889123468085825, mse=56516.8651075948
Iteration: 12, B1=5.889315500677202, mse=56516.86192797619
Iteration: 13, B1=5.889454622144611, mse=56516.860259141016
Iteration: 14, B1=5.889555411195862, mse=56516.859383246396
Iteration: 15, B1=5.889628429638243, mse=56516.858923529704
Iteration: 16, B1=5.8896813291631585, mse=56516.85868224562
Iter

Таким образом, методом GD коэффициент **B1** получается тоже **5.890**

**Задание 3***: Произвести вычисления как в пункте 2, но с вычислением intercept. Учесть, что изменение коэффициентов должно производиться
на каждом шаге одновременно (то есть изменение одного коэффициента не должно влиять на изменение другого во время одной итерации).

**Решение:**

In [23]:
B0 = 0.1

In [24]:
# for i in range(100):
#   # B0 = B0 - alpha * (2 / n) * np.sum(B1 * zp + B0 - ks),
#   B1 =  B1 - alpha * (2 / n) * np.sum((B1 * zp + B0 - ks) * zp)
#   if i % 10 == 0:
#     print(f'Iteration: {i}, B0={B0}, B1={B1}, mse={mse(B1)}')
B0 = 0.1
B1 = 0.1
all_mse = []
for i in range(5000000):
    predicted = B0 + B1 * zp
    residuals = ks - predicted
    all_mse.append(np.sum(residuals**2))
    B0, B1 = B0 - alpha * ((2/n) * np.sum(residuals) * -1), B1 - alpha * ((2/n) * residuals.dot(zp) * -1)
    if i % 100000 == 0:
      print(f'Iteration: {i}, B0={B0}, B1={B1}, mse={all_mse[i]}')

Iteration: 0, B0=0.1139932, B1=1.6950780000000003, mse=5181963.839999999
Iteration: 100000, B0=176.8178756872003, B1=4.588422873669245, mse=246028.4307165328
Iteration: 200000, B0=283.1956881772298, B1=3.805435141410038, mse=130442.31858939507
Iteration: 300000, B0=347.24756959921285, B1=3.333984975180734, mse=88537.1755817341
Iteration: 400000, B0=385.81429152799836, B1=3.0501168504576253, mse=73344.6842159825
Iteration: 500000, B0=409.0359675710964, B1=2.8791950478876815, mse=67836.72559008695
Iteration: 600000, B0=423.0181317603773, B1=2.77628013758468, mse=65839.84387436202
Iteration: 700000, B0=431.43702947702633, B1=2.7143133281816474, mse=65115.8848075947
Iteration: 800000, B0=436.5061902881824, B1=2.6770020634068015, mse=64853.41721813871
Iteration: 900000, B0=439.55841781180436, B1=2.6545363193656377, mse=64758.26096044827
Iteration: 1000000, B0=441.396215664944, B1=2.6410093146294247, mse=64723.76255477298
Iteration: 1100000, B0=442.502784858385, B1=2.632864476123355, mse=647

Таким образом, методом градиентного спуска мы нашли *те же самые* коэффициенты линейной регрессии: **B0 = 444.177** и **B1 = 2.621**. Получившаяся зависимость:
 ***y = 444.177 + x * 2.621***